In [1]:
# Required imports
import pandas as pd  # For storing and manipulation of data files
import numpy as np  
from sklearn.ensemble import RandomForestClassifier # For finding the weights of features
import matplotlib.pyplot as plt  # For plotting figures
import seaborn as sns # To make figures more clear
from sklearn.feature_selection import SelectFromModel # For selecting the important features
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score # metric for model performance

from sklearn.model_selection  import train_test_split
import sklearn.model_selection   # For searching best parameter
from sklearn.metrics import classification_report


%matplotlib inline

In [2]:
# Reading train and test data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
# selecting the train labels
train_labels = train_data.TARGET
train_ID = train_data.ID
# Removing ID and target from the train data
train_features = train_data.drop(labels=["ID","TARGET"],axis = 1)

test_ID = test_data.ID
test_data = test_data.drop("ID",axis = 1)

In [4]:
print(train_features.shape)
print(train_labels.shape)
print(test_data.shape)

(76020, 369)
(76020,)
(75818, 369)


In [5]:
def select_features(data,labels = []):
    if len(labels) == 0 :
        return select_features(train_features,train_labels)
    else:
        rf = RandomForestClassifier(n_estimators=5,random_state=50)
        sfm = SelectFromModel(rf, threshold=0.05)
        sfm.fit(data,labels)
        n_features = np.sum(sfm.get_support())

        while n_features < 10:
            sfm.threshold -= 0.004
            n_features = np.sum(sfm.get_support())
            features_selected = train_features.columns[sfm.get_support()]
        print("No of features selected : ",n_features)
        print ("Features selected : ",features_selected)
        return sfm.get_support()

In [6]:
def preprocessing(data_features,labels = []):
    
#     Checking if there are any duplicate rows
#     data.drop_duplicates(inplace = True)

    # There are 370 columns in the data.
    print(data_features.shape)
    
    # Feature Selection
    # Selecting the best features from Model
    features_selected = select_features(data_features,labels)
    # plotting scatter matrix for the selected features
    data_selected_features = data_features[data_features.columns[features_selected]]
    data_selected_features.describe()
#   pd.scatter_matrix(train_selected_features, alpha = 0.3, diagonal = 'kde');
    # Adding 1 to avoid infinite values
    log_data = np.log(data_selected_features.add(1))
    display(log_data.describe())
    # We can see from the data
    # Normalize each feature of the data
    normalized_data = pd.DataFrame(normalize(log_data.fillna(0),axis = 0))
    # PCA
    pca = PCA(n_components=2)
    data_projected = pca.fit_transform(normalized_data)
    # Visualize data 
    return data_projected

In [7]:
train_projected_data = preprocessing(train_features,train_labels)

(76020, 369)
('No of features selected : ', 10)
('Features selected : ', Index([u'var15', u'saldo_var5', u'saldo_var30', u'num_var45_hace2',
       u'num_var45_hace3', u'num_var45_ult3', u'saldo_medio_var5_hace3',
       u'saldo_medio_var5_ult1', u'saldo_medio_var5_ult3', u'var38'],
      dtype='object'))


C:\Users\priyaranjanjc\AppData\Local\Continuum\Anaconda2\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,var15,saldo_var5,saldo_var30,num_var45_hace2,num_var45_hace3,num_var45_ult3,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,var38
count,76020.000000,75920.000000,75907.000000,76020.000000,76020.000000,76020.000000,76016.000000,75972.000000,75996.000000,76020.000000
mean,3.473973,2.476382,3.436064,0.832833,0.720584,1.353765,1.662790,2.587270,2.581740,11.482260
std,0.328380,2.703187,3.609487,1.232658,1.116570,1.557871,2.364744,2.737448,2.769442,0.560583
min,1.791759,-1.660731,-1.660731,0.000000,0.000000,0.000000,0.000000,-0.127833,0.000000,8.549612
25%,3.178054,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,11.125373
50%,3.367296,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,11.575056
75%,3.713572,NaN,NaN,1.386294,1.386294,2.564949,NaN,NaN,NaN,11.684837
max,4.663439,13.336394,15.056224,5.837730,5.828946,6.687109,14.248812,13.307065,13.207378,16.908131


In [8]:
test_projected_data = preprocessing(test_data,[])

(75818, 369)


MemoryError: 

In [ ]:
data = train_projected_data
lables = train_labels